In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.core.constants import ARC_LAMPS
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60148)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/raw_metadata.hdf5
[INFO]: found 1545 frames in store '/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/raw_metadata.hdf5'
[INFO]: number of frames after filtering 1545
[INFO]: total number of frames found 1545


In [3]:
masters_mjd = qdrp.get_master_mjd(frames_table.mjd.min())
masters_path = os.path.join(ORIG_MASTER_DIR, str(masters_mjd))
print(masters_path)

/home/mejia/Research/lvm/lvmdata/calib/60142


In [4]:
lamps = [lamp.lower() for lamp in ARC_LAMPS]

arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

marc_paths = []
for lamp in lamps:
    rest_lamps = copy(lamps)
    rest_lamps.remove(lamp)
    
    lamp_query = f"{lamp} & not " + " & not ".join(rest_lamps)

    for camera in arc_analogs.groups:
        arcs = arc_analogs.get_group(camera)
        arcs = arcs.query(lamp_query)
        arc = arcs.iloc[0].to_dict()
        print(arcs.to_string())

        # define master paths
        mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
        # define master frame path
        marc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"marc_{lamp}", camera=arc["camera"])
        warc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"wmarc_{lamp}", camera=camera)
        marc_paths.append(marc_path)
        os.makedirs(os.path.dirname(marc_path), exist_ok=True)
        
        # preprocess and detrend arcs
        darc_paths = []
        for arc in arcs.to_dict("records"):
            darc_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=arc["imagetyp"], **arc)
            darc_paths.append(darc_path)
    
        # create master arc (2D image)
        imageMethod.create_master_frame(in_images=darc_paths, out_image=marc_path, master_mjd=masters_mjd)
        
        # extract combined (master) arc
        mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
        imageMethod.extract_spectra(in_image=marc_path, out_rss=warc_path, in_trace=mtrace_path, method="aperture", aperture=3)

[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001573.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001572.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001588.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001575.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001574.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b1-00001589.fits


     hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
21      s    1111  60148  60148      arc  sp1     b1    1573     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001573.fits
256     s    1111  60148  60148      arc  sp1     b1    1572     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001572.fits
1145    s    1111  60148  60148      arc  sp1     b1    1588     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001588.fits
1172    s    1111  60148  60148      arc  sp1     b1    1575    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00001575.fits
1274    s    1111  60148  60148      arc  sp1     b1    1574    150.0 

[INFO]: combining 6 frames into master frame
[INFO]: writing master frame to 'lvm-marc_neon-b1.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001575.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001573.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001589.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001588.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001574.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b2-00001572.fits


     hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
12      s    1111  60148  60148      arc  sp2     b2    1575    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001575.fits
248     s    1111  60148  60148      arc  sp2     b2    1573     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001573.fits
358     s    1111  60148  60148      arc  sp2     b2    1589     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001589.fits
450     s    1111  60148  60148      arc  sp2     b2    1588     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00001588.fits
1009    s    1111  60148  60148      arc  sp2     b2    1574    150.0 

[INFO]: combining 6 frames into master frame
[INFO]: writing master frame to 'lvm-marc_neon-b2.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001574.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001573.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001572.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001589.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001575.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-b3-00001588.fits


...

[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z1.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-z2-00001581.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-z2-00001580.fits


     hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
1253    s    1111  60148  60148      arc  sp2     z2    1581    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001581.fits
1319    s    1111  60148  60148      arc  sp2     z2    1580    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z2-00001580.fits


[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z2.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-z3-00001580.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60148/ancillary/lvm-darc-z3-00001581.fits


     hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
101     s    1111  60148  60148      arc  sp3     z3    1580    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001580.fits
1465    s    1111  60148  60148      arc  sp3     z3    1581    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z3-00001581.fits


[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z3.fits'
